In [1]:
%matplotlib inline 

# Regression
***

## Table of Contents
***
* [Aim](#aim)
* [Code Setup](#code-setup)
* [Data Import](#data-import)

## Aim <a class="anchor" id="aim"></a>
***

The aim of this notebook is to perform Regression on the Instacart Data

## Code Setup <a class="anchor" id="code-setup"></a>
***

### Constants

In [2]:
instacart_dir = "instacart_2017_05_01/"
my_orders_file = "my_orders.csv"
products_file = "products.csv"
aisles_file = "aisles.csv"
departments_file = "departments.csv"
orders_products_prior_file = "my_order_products__prior.csv"
orders_products_train_file = "my_order_products__train.csv"

### Import Libraries

In [3]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("paper")
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [4]:
def read_csv_file(file_name):
    return pd.read_csv(instacart_dir + file_name)

### Import the Data

In [5]:
orders = read_csv_file(my_orders_file)
products = read_csv_file(products_file)
aisles = read_csv_file(aisles_file)
orders_products_prior = read_csv_file(orders_products_prior_file)
orders_products_train = read_csv_file(orders_products_train_file)
orders_products = pd.concat([orders_products_prior, orders_products_train])

In [6]:
def join_products(train, test):
    train_orders_prod = train.merge(orders_products, on="order_id")
    train_orders_prod = train_orders_prod.merge(products, on="product_id")

    test_orders_prod = test.merge(orders_products, on="order_id")
    test_orders_prod = test_orders_prod.merge(products, on="product_id")
    
    return (train_orders_prod, test_orders_prod)

In [7]:
def add_days_to_next_order(order_df):
    orders = order_df.copy()
    orders["days_to_next_order"] = np.nan
    for index, cur_order in orders.iterrows():
        next_order_number = cur_order.order_number + 1
        user_id = cur_order.user_id
        next_order_df = orders.query("user_id == " + str(user_id) + " and  order_number == " + str(next_order_number))
        if(next_order_df.shape[0] == 1):
            next_order = next_order_df.iloc[0]
            days_to_next_order = next_order.days_since_prior_order
            if(np.isfinite(days_to_next_order)):
                orders.loc[index, 'days_to_next_order'] = days_to_next_order
    return orders

In [8]:
def key_columns(next_one):
    list = ["order_id", "days_to_next_order", next_one]
    return list

In [9]:
def split_data_to_features_and_value(df_train, df_test):
    # Target
    y_train = df_train['days_to_next_order'].copy()
    X_train = df_train.copy() 
    # Feature
    X_train.drop(['days_to_next_order'], axis=1, inplace=True)

    # Testing
    y_test = df_test['days_to_next_order'].copy()
    X_test = df_test.copy() 
    # Feature
    X_test.drop(['days_to_next_order'], axis=1, inplace=True)
    return (X_train, y_train, X_test, y_test)

In [10]:
def add_each_aisle_to_df(df):
    df_aisles = df.copy().merge(aisles, on="aisle_id")
    for index, row in aisles.iterrows():
        aisle_name = row.aisle
        df_aisles[aisle_name] = 0
    return df_aisles

In [11]:
def add_each_product_to_df(df, popular_products):
    df_products = df.copy().merge(products, on="product_id")
    for index, row in popular_products.iterrows():
        product_name = row.product_name
        df_products[product_name] = 0
    return df_products

In [12]:
def group_by_order_in_df(df_ordered, item_name, cols_to_drop):
    # Create new df with only the column names
    df = df_ordered.copy()
    df.drop(df.index, inplace=True)
    order_ids = df_ordered.order_id.unique().tolist()
    for order_id in order_ids:
        temp_df = df_ordered.query("order_id == " + str(order_id))
        # Need a copy as not to modify something we are iterarting over
        first_row = temp_df.copy().head(1)
        for i, temp_row in temp_df.iterrows():
            temp_name = temp_row[item_name]
            # If the feature is present, set it to 1
            first_row[temp_name] = 1
        # Add the aggregated entry
        df = pd.concat([df, first_row])
    df = df.drop(cols_to_drop, axis=1)
    return df    

In [13]:
def find_best_feature(X, y, X_test, y_test):
    n_features = X.shape[1]
    # Not zero as some negative scores can happen
    max_score = -100000
    max_feature = None
    for i in range(n_features):
        feature = X.columns[i]
        X_subset = X[[feature]]
        X_test_subset = X_test[[feature]]
        model = linear_model.LinearRegression()
        model.fit(X_subset, y)
        # R squared metric
        score = model.score(X_test_subset, y_test)
        if score > max_score:
            max_score = score
            max_feature = feature
    if max_feature is None :
        print("Issue, max feature not found, size", n_features)
        print(X.columns.values)
    return (max_score, max_feature)

In [14]:
# This is a very brute force way computing all possibilities of finding the solution with the highest score
def order_best_features_and_score(X_train, y_train, X_test, y_test):
    n_features = X_train.shape[1]
    model_scores = [0]
    best_features = []
    best_score = -10000
    X_selection = X_train.copy()
    X_test_selection = X_test.copy()
    X_subsets = pd.DataFrame(index=X_train.index)
    X_test_subsets = pd.DataFrame(index=X_test.index)

    for i in range(n_features):
        max_score, next_best_feature = find_best_feature(X_selection, y_train, X_test_selection, y_test)

        X_subset = X_selection[[next_best_feature]]
        X_test_subset = X_test_selection[[next_best_feature]]
    
        X_subsets[next_best_feature] = X_subset
        X_test_subsets[next_best_feature] = X_test_subset
        
        model = linear_model.LinearRegression()
        model.fit(X_subsets, y_train)
        score = model.score(X_test_subsets, y_test)
        model_scores.append(score)
        
        score_increase_threshold = 0.01
        if score > (best_score + score_increase_threshold):
            best_score = score
            best_features = X_subsets.columns.values

        X_selection.drop([next_best_feature], axis=1, inplace=True)
        X_test_selection.drop([next_best_feature], axis=1, inplace=True)
    return (best_features, best_score, model_scores)

#### Test and training split for all users

In [15]:
test_percentage = 0.2
orders = add_days_to_next_order(orders)
# Can use this to estimate what would be ordered
orders_with_no_next_order = orders[orders['days_to_next_order'].isnull()]
# Can split this into train and test
orders_with_next_order = orders[~orders['days_to_next_order'].isnull()]
train_orders, test_orders = train_test_split(orders_with_next_order, test_size=test_percentage)
print("Training size is :", train_orders.shape[0])
print("Testing size is :", test_orders.shape[0])

Training size is : 228
Testing size is : 58


#### Test and training split for specific user

In [16]:
count_orders = (orders
                .groupby("user_id")
                .size()
                .reset_index(name="count")
                .sort_values(by=['count'], ascending=False)
               )
# Get the most frequent user as this will have the most data associated with it
most_frequeny_user_id =  count_orders.iloc[0].user_id
single_user_orders = orders.query("user_id == " + str(most_frequeny_user_id))
single_user_orders = add_days_to_next_order(single_user_orders)
user_orders_with_next_order = single_user_orders[~single_user_orders['days_to_next_order'].isnull()]
user_train_orders, user_test_orders = train_test_split(user_orders_with_next_order, test_size=test_percentage)
print("User id for single investigation :", most_frequeny_user_id)
print("Training size for single user is :", user_train_orders.shape[0])
print("Testing size for single user is is :", user_test_orders.shape[0])

User id for single investigation : 47562
Training size for single user is : 70
Testing size for single user is is : 18


### Merge Product and Aisle to the data

In [17]:
train_orders_prod, test_orders_prod = join_products(train_orders, test_orders)
train_orders_prod.head()

,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_to_next_order,Unnamed: 0_y,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2066017,3312511,124199,prior,30,5,0,2.0,9.0,31405680,6656,1,1,Organic California Style Sprouted Bread,112,3
1,2066018,657645,124199,prior,31,0,12,9.0,8.0,6230770,6656,7,1,Organic California Style Sprouted Bread,112,3
2,2066002,1730739,124199,prior,15,6,11,21.0,7.0,16406557,6656,3,1,Organic California Style Sprouted Bread,112,3
3,2066019,690551,124199,prior,32,1,7,8.0,1.0,6542765,6656,5,1,Organic California Style Sprouted Bread,112,3
4,2066030,695010,124199,prior,43,6,13,8.0,14.0,6584241,6656,1,1,Organic California Style Sprouted Bread,112,3


## Product Investigation
***

In [18]:
key_cols = key_columns("product_id")
wanted_info_train = train_orders_prod[key_cols]
wanted_info_test = test_orders_prod[key_cols]

wanted_info_train.head()

,order_id,days_to_next_order,product_id
0,3312511,9.0,6656
1,657645,8.0,6656
2,1730739,7.0,6656
3,690551,1.0,6656
4,695010,14.0,6656


### Add Product Columns

In [19]:
# Only doing what products are currenty present in training set
popular_products_ids = wanted_info_train.product_id.unique().tolist()
popular_products = products[products['product_id'].isin(popular_products_ids)]
df_with_product_info_train = add_each_product_to_df(wanted_info_train, popular_products)
df_with_product_info_test = add_each_product_to_df(wanted_info_test, popular_products)

df_with_product_info_train.head()

,order_id,days_to_next_order,product_id,product_name,aisle_id,department_id,Cantaloupe,Minis Original Saltine Crackers,Cheerios Cereal,Dark Red Kidney Beans,...,Peppermint Caffeine Free Herbal Tea - 40 CT,Organic Red Cabbage,Breakfast Burrito,Organic Half & Half,Organic Purely Peppermint Tea Bags 16 Count,Vegetable Broth Low Sodium,Organic Whole Bean Coffee,White Grape Juice Cocktail,Feta Crumbles,Cucumber Kirby
0,3312511,9.0,6656,Organic California Style Sprouted Bread,112,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,657645,8.0,6656,Organic California Style Sprouted Bread,112,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1730739,7.0,6656,Organic California Style Sprouted Bread,112,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,690551,1.0,6656,Organic California Style Sprouted Bread,112,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,695010,14.0,6656,Organic California Style Sprouted Bread,112,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
cols_to_drop = ["product_id", "product_name", "order_id", "department_id", "aisle_id"]
item = "product_name"
df_prod_group_train = group_by_order_in_df(df_with_product_info_train, item, cols_to_drop)
df_prod_group_test = group_by_order_in_df(df_with_product_info_test, item, cols_to_drop)

df_prod_group_train.head()

,days_to_next_order,Cantaloupe,Minis Original Saltine Crackers,Cheerios Cereal,Dark Red Kidney Beans,Vanilla Almond Breeze Almond Milk,Organic Diced Tomatoes,Extra Dark Chocolate Baking Chips,100% Oatnut Bread,Jack's Special Salsa Medium Hot,...,Peppermint Caffeine Free Herbal Tea - 40 CT,Organic Red Cabbage,Breakfast Burrito,Organic Half & Half,Organic Purely Peppermint Tea Bags 16 Count,Vegetable Broth Low Sodium,Organic Whole Bean Coffee,White Grape Juice Cocktail,Feta Crumbles,Cucumber Kirby
0,9.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,8.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,7.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X_train, y_train, X_test, y_test = split_data_to_features_and_value(df_prod_group_train, df_prod_group_test)
# Where X are the features and Y is the days to next order (value trying to predict)
X_train.head()

,Cantaloupe,Minis Original Saltine Crackers,Cheerios Cereal,Dark Red Kidney Beans,Vanilla Almond Breeze Almond Milk,Organic Diced Tomatoes,Extra Dark Chocolate Baking Chips,100% Oatnut Bread,Jack's Special Salsa Medium Hot,"Noodle Soup, Shin Cup, Gourmet Spicy",...,Peppermint Caffeine Free Herbal Tea - 40 CT,Organic Red Cabbage,Breakfast Burrito,Organic Half & Half,Organic Purely Peppermint Tea Bags 16 Count,Vegetable Broth Low Sodium,Organic Whole Bean Coffee,White Grape Juice Cocktail,Feta Crumbles,Cucumber Kirby
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
model.intercept_

-115244503188.12718

In [ ]:
find_best_feature(X_train, y_train, X_test, y_test)

(0.11112188609451434, '2% Reduced Fat Organic Milk')

In [ ]:
best_features, best_score, model_scores = order_best_features_and_score(X_train, y_train, X_test, y_test)
print("Best Score :", best_score)
print(best_features)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

In [ ]:
nbfs, nbs, model_scores = order_best_features_and_score(X_train[best_features], y_train, X_test[best_features], y_test)
print("Best Score :", nbs)
print(nbfs)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

## Aisle Rollup
***

### Extract Desired Columns

In [ ]:
key_cols = key_columns("aisle_id")
wanted_info_train = train_orders_prod[key_cols]
wanted_info_test = test_orders_prod[key_cols]

wanted_info_train.head()

### Add the Aisle Columns

In [ ]:
df_with_aisle_info_train = add_each_aisle_to_df(wanted_info_train)
df_with_aisle_info_test = add_each_aisle_to_df(wanted_info_test)

df_with_aisle_info_train.head()

### Populate the Aisle Columns

In [ ]:
# Aggregates the data so we have days to next order with the aisles required checked
cols_to_drop = ["aisle", "aisle_id", "order_id"]
item = "aisle"
df_aisle_group_train = group_by_order_in_df(df_with_aisle_info_train, item, cols_to_drop)
df_aisle_group_test = group_by_order_in_df(df_with_aisle_info_test, item, cols_to_drop)

df_aisle_group_train.head()

### Preparing the data for modelling

In [ ]:
X_train, y_train, X_test, y_test = split_data_to_features_and_value(df_aisle_group_train, df_aisle_group_test)
# Where X are the features and Y is the days to next order (value trying to predict)
X_train.head()

### Generate the Model

In [ ]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
model.intercept_

### Find the best feature/aisle

In [ ]:
find_best_feature(X_train, y_train, X_test, y_test)

### Find best features in Order and calculate scores

In [ ]:
best_features, best_score, model_scores = order_best_features_and_score(X_train, y_train, X_test, y_test)
print("Best Score :", best_score)
print(best_features)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

In [ ]:
nbfs, nbs, model_scores = order_best_features_and_score(X_train[best_features], y_train, X_test[best_features], y_test)
print("Best Score :", nbs)
print(nbfs)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

In [ ]:
nnbfs, nnbs, model_scores = order_best_features_and_score(X_train[best_features], y_train, X_train[best_features], y_train)
print("New Best Score :", nnbs)
print(nnbfs)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

### Specific User Investigation

In [ ]:
# For the User
train_orders_prod, test_orders_prod = join_products(user_train_orders, user_test_orders)
wanted_info_train = train_orders_prod[key_cols]
wanted_info_test = test_orders_prod[key_cols]
df_with_aisle_info_train = add_each_aisle_to_df(wanted_info_train)
df_with_aisle_info_test = add_each_aisle_to_df(wanted_info_test)
cols_to_drop = ["aisle", "aisle_id", "order_id"]
item = "aisle"
df_aisle_group_train = group_by_order_in_df(df_with_aisle_info_train, item, cols_to_drop)
df_aisle_group_test = group_by_order_in_df(df_with_aisle_info_test, item, cols_to_drop)
X_train, y_train, X_test, y_test = split_data_to_features_and_value(df_aisle_group_train, df_aisle_group_test)
best_features, best_score, model_scores = order_best_features_and_score(X_train, y_train, X_test, y_test)
print("Best Score :", best_score)
print(best_features)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

In [ ]:
best_features, best_score, model_scores = order_best_features_and_score(X_train[best_features], y_train, X_test[best_features], y_test)
print("Best Score :", best_score)
print(best_features)

In [ ]:
plt.title("Score versus features")
plt.xlabel('Number of features') 
plt.ylabel('R squared')
plt.plot(model_scores)
plt.show()

In [ ]:
# NOW WE WANT TO USE PCA/REGULARISATION AND REMOVE SOME UNNEEDED INFO/AISLES

In [ ]:
# DONT FORGET TO MEAN CENTRE